In [ ]:
import requests
from bs4 import BeautifulSoup



def robots_context(text):
    result_dict = {}
    lines = text.split('\n')
    sitemaps = []
    for line in lines:
        if line.startswith('Sitemap:'):
            #včasih ni napisano nič za "Sitemap:"
            if len(line.split())>1: sitemaps.append(line.split()[1])
        elif line.startswith('User-agent:') and line.split()[1] == '*':
            disallows = []
            allows = []
            for agent in lines[lines.index(line)+1:]:
                print(agent)
                #če ni nič od naštetega v vrsici, nima smisla nadaljevati
                if agent.strip() and  not agent.strip().startswith(('Disallow:', 'Allow:', '#')):
                    break
                if agent.startswith('Disallow:'):
                    if len(agent.split())>1: disallows.append(agent.split()[1])        
                if agent.startswith('Allow:'):
                    if len(agent.split())>1: allows.append(agent.split()[1]) 
            result_dict['User-agent'] = {'Disallow': disallows, 'Allow': allows}
        elif line.startswith('Crawl-delay:'):
            if len(line.split())>1: result_dict['Crawl-delay'] = line.split()[1]
        else:
            continue
    result_dict['Sitemap'] = sitemaps
    return result_dict

def main_page():
    #vstavimo neko stran ki ni prva.

    #kliče funkcijo, ki scrapa stran

    #pridobi osnovno stran in vstavi dobljene info v bazo

    #na izhodu dobimo:
    #id glavne -> mogoče samo to vrne, ker se lahko kliče bazo
    #vse strani iz sitemap
    pass



def url_sitemap(url):
    #za vstavlen url osnovne domene dobi vse povezave na strani
    response = requests.get(url + '/robots.txt')
    sitemap_list = robots_context(response.text)
    print(sitemap_list['Sitemap'])
    urls = []
    for sitemap in sitemap_list['Sitemap']:
        urls.extend(parse_sitemap(sitemap))
    return urls


def parse_sitemap(url_sitemap):
    #dobi urlje od sitmap xml datoteke
    urls = []

    #če je ni
    if url_sitemap is  None:
        return []

    # Fetch the sitemap XML file
    response = requests.get(url_sitemap)
    if response.status_code == 200:
        #preoblikujemo v leš
        soup = BeautifulSoup(response.content, 'xml')
        

    
        #išče <sitemap>, ki nakazuje na .xml datoteke
        sitemap = soup.find('sitemap')
        if sitemap is not None:
            
            
            for sitemap in soup.sitemapindex.find_all('sitemap'):
                # Extract the URLs from the sitemap
                loc = sitemap.loc.text
                
                #print(f'Location: {loc}')

                #če so xml datoteke
                if loc.endswith(".xml") or loc.endswith(".XML"):
                    urls.extend(parse_sitemap(loc))
                    
            #kliče sebe s parametrom, ki vsebuje ime xml datoteke z url-ji
        
        else:
            #datotka vsebuje url-je, išče <urlset>
            urlset = soup.find('urlset')
            if urlset is not None:
                url = soup.find('url')
                if url is not None:
                    urls.extend([element.text for element in soup.find_all('loc')])
       
  
    return urls

In [ ]:
#pomembno ker nekateri strani brez tega ne dovolijo dostop!
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

url = 'https://okusno.je/'  # Replace example.com with the actual domain
response = requests.get(url, headers=headers)
if response.status_code == 200:  # Check if the response is successful
    #print(len(url_sitemap(url))) #strani je preveč da bi jih editor izpisal zato len
    resp = requests.get(url + '/robots.txt', headers=headers)
    print(robots_context(resp.text))
else:
    print('Failed to retrieve robots.txt')